<a href="https://colab.research.google.com/github/joankl/Solar-Neutrinos-Project/blob/main/Solar-Neutrinos-Project%20/bis_data_candidates/lifetime_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Dedicated to Compute the Dataset Raw Lifetime using the runID lifetime calculator

The Lifetime should be calculated using the entery range of runIDs used for the analysis! No need to account for cuts, we are juts interested in the total lifetime of the available dataset. The cuts will be accountes in the selection efficiency based on MC.

In [1]:
import numpy as np
import pandas as pd
import glob
import pickle

# Load Files

## Load atmospheric and HS results

In [3]:
# ==== Define the removal time in seconds ====
hs_remove_time = 1
atm_remove_time = 20

# ====== Load Dictionary ======

# Data directory pattern
main_dir = '/home/joankl/data/solars/real_data/bisMSB/bkg_candidates/Analysis*/'

# Dictionary name list to load
dict_name_list = ['hs_dict', 'atm_dict']

# Create an general empty dictionary to save the loaded dictionaries with names of dict_name_list
general_dict = {}

# Create a list with the dictionaries to be iterated
#dict_list = [hs_dict, atm_dict, prompt_coinc_dict, delay_coinc_dict]

# Iterate over each dictionary
for dict_name_list_i in dict_name_list:

    # Create list of directories for the dict_i
    file_dir_list = glob.glob(main_dir + dict_name_list_i + '.pkl')

    # Create temporal dictionary to save the data in file_dir_list. Only refresh when dict_i changes
    temp_dir = {}

    add_dict_key_ctrl = 0  # Counter to controls when keys are added to temp_dir.

    # Iterare over the directory files
    for fdir_i in file_dir_list:
        
        with open(fdir_i, 'rb') as f:
            dict_i = pickle.load(f)

        # Add the keys of dict_i to temp_dir. It will create the keys of the loaded dictionary one time since add_dict_key_ctrl will be > 0.
        if add_dict_key_ctrl == 0:
            dict_keys = list(dict_i.keys())
            for key_i in dict_keys:
                temp_dir[key_i] = []
            
            add_dict_key_ctrl += 1

        # Append the values of the loaded dict_i to the tempo dict
        for key_i in dict_keys:
            temp_dir[key_i].append(dict_i[key_i])

        # Once the file_dir_list is fully iterated, create the genara_dict keys and save the temp_dir informarion
        general_dict[dict_name_list_i] = temp_dir

# ====== Extract Data ======
hs_dict = general_dict['hs_dict']
atm_dict = general_dict['atm_dict']


hs_counter = hs_dict['counter']
atm_counter = atm_dict['counter']

total_hs = int(np.sum(hs_counter))
total_atm = int(np.sum(atm_counter))

## Load the livetime list

In [11]:
#main_dir = 'C:/Users/joanc/jupyter notebooks/solar neutrino analysis/bis_data_candidates/run_lifetime_lists/*'
main_dir = 'run_lifetime_lists/*'

flist = glob.glob(main_dir)

runID_list, time1_lt, time2_lt = [], [], []

for fdir in flist:

  with open(fdir, "r", encoding="utf-8") as f:
      for linea in f:
          if linea.strip():  # Evitar líneas vacías
              valores = linea.split()  # Separa por espacios o tabulaciones
              runID_list.append(int(valores[0]))
              time1_lt.append(float(valores[1]))
              time2_lt.append(float(valores[2]))

runID_list = np.array(runID_list)
time1_lt = np.array(time1_lt) # RAW LIVETIME (DAYS)
time2_lt = np.array(time2_lt) # NET LIVETIME (DAYS)

# RUN SELECTION:
#runID_selection = (runID_lt >= 300700)

#runID_lt = runID_lt[runID_selection]
#time1_lt = np.array(time1_lt)[runID_selection] # RAW LIVETIME (DAYS)
#time2_lt = np.array(time2_lt)[runID_selection] # NET LIVETIME (DAYS)

## Load the Full Dataset runIDs

In [34]:
main_fdir = '/home/joankl/data/solars/real_data/bisMSB/first_candidates/analysis*/resume_files/'
fdir_list = glob.glob(main_fdir)

# Define the observables to grab from dataset: Save the into a dictionary
obs_list = ['runID', 'posr_av', 'energy_corrected']
obs_dict = {obs_i: np.array([]) for obs_i in obs_list}

for fdir_i in fdir_list:
    for var_i in obs_list:
        obs_i = np.load(fdir_i + var_i +'.npy')
        obs_dict[var_i] = np.append(obs_dict[var_i], obs_i)


runID_dataset = obs_dict['runID']
runID_dataset = np.unique(runID_dataset)

print(f'min. runID dataset: {np.min(runID_dataset)}')
print(f'max. runID dataset: {np.max(runID_dataset)}')

min. runID dataset: 354099.0
max. runID dataset: 371216.0


# Compute the Dataset Livetime

In [35]:
# Extract the net lifetime (time2_lt) where runID from data is in runID_list
same_runID_condition = np.isin(runID_list, runID_dataset)

filtered_lt = time2_lt[same_runID_condition]
tot_dataset_lt_days = np.sum(filtered_lt)

print(f'All dataset with livetime = {tot_dataset_lt_days} days')

All dataset with livetime = 322.73863212103015 days


In [ ]:
# Compute the time to removed due to atm and hs (in days)

atm_time = total_atm * (atm_remove_time) / (24 * 3600)
hs_time = total_hs * (hs_remove_time) / (24 * 3600)

print(f'Time removed due to atmospherics = {atm_time} days')